In [1]:
#imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler as std
from sklearn.preprocessing import Normalizer as nr
from sklearn import preprocessing
from sklearn.decomposition import PCA
import os
import gc
import sys
import torch 
import torch.nn as nn
import keras
import datetime
from tqdm import tqdm 
import gc 
import pickle as p

In [2]:
df=pd.read_csv("/kaggle/input/cleaned-weld-right-data/cleaned_data.csv")
A=df

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.Production.unique(),df["Order Operation No"].unique()
le = preprocessing.LabelEncoder()
col=["Production","Order Operation No"]

df[col[0]]=df[col[0]].values.astype(np.str_)
df[col[1]]=df[col[1]].values.astype(np.str_)
df[col[0]] = le.fit_transform(df[col[0]].values)
df[col[1]] = le.fit_transform(df[col[1]].values)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827534 entries, 0 to 827533
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Employee Code       827534 non-null  int64  
 1   Machine             827534 non-null  object 
 2   Production          827534 non-null  int64  
 3   Order Operation No  827534 non-null  int64  
 4   Date                827534 non-null  object 
 5   Time                827534 non-null  object 
 6   Current             827531 non-null  float64
 7   Humidity            827534 non-null  int64  
 8   Temperature         827534 non-null  int64  
 9   Flow                827534 non-null  float64
 10  Job Temp            827534 non-null  float64
 11  Voltage             827534 non-null  float64
 12  Defect              827534 non-null  object 
 13  label               827534 non-null  int64  
dtypes: float64(4), int64(6), object(4)
memory usage: 88.4+ MB


In [4]:
col_me=[i for i in df.columns]
col_me

['Employee Code',
 'Machine',
 'Production',
 'Order Operation No',
 'Date',
 'Time',
 'Current',
 'Humidity',
 'Temperature',
 'Flow',
 'Job Temp',
 'Voltage',
 'Defect',
 'label']

In [5]:
"""
domain specific features
"""
def feature1(df):
    #df["Voltage"]=df["Voltage"]
    df["Power"]=(df["Current"])*((df["Voltage"])+1)
    df["Delta_Temperature"]= df["Job Temp"]-df["Temperature"]
    df["Resistance"]=(df["Voltage"]+1)/(df["Current"])
    df["Delta_Charge"]=(df["Current"])*(1/1000)*df["Delta_Time"]
    df["Alpha"]=df["Humidity"]/df["Delta_Temperature"]
    df["Heat"]=df["Power"]*df["Delta_Time"]*(1/1000)
    return df
"""
Lag features
"""
def lag(df):
    coll=["Power","Delta_Time","Delta_Temperature","Alpha","Heat",'Current','Humidity','Flow','Job Temp']
    for i in coll:
        temp=[]
        for j in range(len(df[i])):
            if j==0 :
                temp.append(0)
            else:
                temp.append(df[i].iloc[j] - df[i].iloc[j-1] )
        df[f"lag_{i}"]=temp
    return df
""")
pca , ica 
"""
def pca_originals(df):
    """
     to make our ml model stable
        
    """
    names=coll=["Current","Humidity","Temperature","Flow","Job Temp","Voltage"]
    pca= PCA(n_components=None)
    #df[coll]=std_transform(df[coll],coll)
    df[["Pca_"+i for i in coll]]= pca.fit_transform(df[names])
    return df

                

In [6]:
# a=pd.DataFrame({0:[1,2,3],1:[4,5,6]})
# a.head()
# a[[0,1]]=std_transform(a,[0,1])
# a

In [7]:
    def time_feature(df):
        """
        gives time in milli second
        1st = time
        2nd = delta_time
        hr : min : sec : m_sec
        
        """
        a=df.Time
        b=df.Date
        a1=[]
        a2=[]
        current=0
        for i in tqdm(range(len(df))):
            if current == b[i]:
                t=a[i].split(":")
            elif i == 0 or current!=b[i]:
                current=b[i]
                t=[0,0,0,0]
            a1.append(int(t[0])*60*60*1000+int(t[1])*60*1000+int(t[2])*1000+int(t[3]))# hr min sec mili sec
            if i == 0:
                a2.append(0)
            else:
                a2.append(a1[i]-a1[i-1])
        return a1,a2
    
    def remove_feature(df,names):
        """
        remove 1_to_1 features
        
        """
        cols= df.columns
        for i in names:
            del df[i]
        return df
    
    def std_transform(df,names):
        """
        standardize values 
        (before pca)
        
        """
        scale=std()
        df[names]=scale.fit_transform(df[names])
        return df
    
    def divi(df,name):
        """
        type - dictionary 
        keys - unique entries in column
        (to be used with employee code "specifically")
        
        """
        df_code= df.groupby(name)
        df1= {}
        
        for i in df_code:
            df1[i[0]] = i[1]
        return df1
    
    def label(df):
        defs=df.Defect.values
        defs=defs!="No Defect"
        defs=defs*1
        df.Defect=defs
        return df
        
    def prep_training(df, history = 60):
        snips1=[]
        for i in tqdm(range( len(df)) ):
            if df.iloc[i].Defect == 1 and i >= history:
                temp=df.iloc[i-history+1:i+1]
                temp=remove_feature(temp,names=cols[:1])
                temp=remove_feature(temp,names=["Defect","label"])
                snips1.append(temp)
        snips0=[]
        """
        startification of entries
        """
        for i in tqdm(range( len(df)) ):
            if df.iloc[i].Defect == 0 and i >= history and len(snips0)<len(snips1)*2:
                temp=df.iloc[i-history+1:i+1]
                temp=remove_feature(temp,names=cols[:1])
                temp=remove_feature(temp,names=["Defect","label"])
                snips0.append(temp)
        snips=[snips0,snips1]
        return snips

In [8]:
def transform2(df):
    #1st convert time
    (a,b)=time_feature(df)
    df["Time"]=a
    df["Delta_Time"]=b
    df=feature1(df)
    df=lag(df)
    #df=pca_originals(df)
    print(df.head())
    print(1)
    #2nd remove useless cols-> machine / date /  
    col=["Machine","Date"]
    df=remove_feature(df,col)
    print(2)
    #3rd correct labels
    df=label(df)
    print(3)
    return df

In [9]:
A=df

In [10]:
            
def transform1(df):
    #1st convert time
    (a,b)=time_feature(df)
    df["Time"]=a
    df["Delta_Time"]=b
    df=feature1(df)
    df=lag(df)
    #df=pca_originals(df)
    print(df.head())
    print(1)
    #2nd remove useless cols-> machine / date /  
    col=["Machine","Date"]
    df=remove_feature(df,col)
    print(2)
    #3rd correct labels
    df=label(df)
    print(3)
    #4th 9 workers 9 dfs
    df=divi(df,name=df.columns[0])
    keys=df.keys()
    """
    output={int->id : grouped df}
    """
    print(4)
    #5th std all dfs
    for i in tqdm(df):
        df[i]=prep_training(df[i])

    print(5)
    gc.collect()
    return df
"""
first approach 40 ki history with subtle gradient for transition from 0 to the first 1 
then adapt the curve
"""
    
def cols_update(df):
    return [i for i in df.columns]
dff=df
cols=cols_update(dff)
dft=transform1(dff)


        
        

100%|██████████| 827534/827534 [00:06<00:00, 120339.44it/s]


   Employee Code Machine  Production  Order Operation No        Date  \
0         382617  TWLD23           5                   4  10-09-2022   
1         382617  TWLD23           5                   4  10-09-2022   
2         382617  TWLD23           5                   4  10-09-2022   
3         382617  TWLD23           5                   4  10-09-2022   
4         382617  TWLD23           5                   4  10-09-2022   

       Time  Current  Humidity  Temperature  Flow  ...         Heat  \
0         0     1.13        74           23  0.01  ...      0.00000   
1  27149040     1.82        74           23  0.00  ...  49411.25280   
2  27149677     2.91        74           23  0.00  ...      1.85367   
3  27150166     0.04        74           23  0.00  ...      0.01956   
4  27150696     2.51        74           23  0.00  ...      1.33030   

   lag_Power lag_Delta_Time  lag_Delta_Temperature  lag_Alpha     lag_Heat  \
0      0.000              0                    0.0   0.000000 

100%|██████████| 66820/66820 [00:09<00:00, 7194.57it/s]

100%|██████████| 117426/117426 [00:16<00:00, 7056.71it/s]

100%|██████████| 36765/36765 [00:05<00:00, 6895.14it/s]

100%|██████████| 27295/27295 [00:03<00:00, 7257.79it/s]

100%|██████████| 8616/8616 [00:01<00:00, 7313.09it/s]

100%|██████████| 256884/256884 [00:36<00:00, 6967.93it/s]

100%|██████████| 114821/114821 [00:16<00:00, 7051.95it/s]

100%|██████████| 23922/23922 [00:03<00:00, 7014.26it/s]

100%|██████████| 174985/174985 [00:24<00:00, 7041.95it/s]

100%|██████████| 9/9 [03:59<00:00, 26.63s/it]


5


In [11]:
df.columns
dft[0][0]

[       Production  Order Operation No      Time  Current  Humidity  \
 54723          17                   4  44881837     5.47        79   
 54724          17                   4  44882339     5.51        79   
 54725          17                   4  44882854    -0.05        79   
 54726          17                   4  44883449     2.90        80   
 54727          17                   4  44883940     2.29        80   
 54728          17                   4  44884452     3.15        80   
 54729          17                   4  44884952     1.69        79   
 54730          17                   4  44885558     0.96        79   
 54731          17                   4  44886055     5.89        79   
 54732          17                   4  44886653     6.39        79   
 54733          17                   4  44887170     2.74        79   
 54734          17                   4  44887753     4.82        79   
 54735          17                   4  44888250     3.70        79   
 54736

In [12]:
for i in dft:
    print(len(dft[i][0]),len(dft[i][1]),"-->",i)

160 80 --> 0
1722 861 --> 97346
720 360 --> 382617
108 54 --> 383138
48 24 --> 387268
3702 1851 --> 391342
1626 813 --> 394269
364 182 --> 394365
3038 1519 --> 394965


In [13]:
dft[0][1][9].iloc[0]

Production               1.600000e+01
Order Operation No       4.000000e+00
Time                     4.133393e+06
Current                  6.770000e+00
Humidity                 8.100000e+01
Temperature              2.400000e+01
Flow                     0.000000e+00
Job Temp                 2.790000e+01
Voltage                  0.000000e+00
Delta_Time               5.110000e+02
Power                    6.770000e+00
Delta_Temperature        3.900000e+00
Resistance               1.477105e-01
Delta_Charge             3.459470e+00
Alpha                    2.076923e+01
Heat                     3.459470e+00
lag_Power                4.540000e+00
lag_Delta_Time          -8.600000e+01
lag_Delta_Temperature    0.000000e+00
lag_Alpha                0.000000e+00
lag_Heat                 2.128160e+00
lag_Current              4.540000e+00
lag_Humidity             0.000000e+00
lag_Flow                 0.000000e+00
lag_Job Temp             0.000000e+00
Name: 110263, dtype: float64

In [14]:
file=open("9_fold_data.bin","wb+")
p.dump(dft,file)
print("file saved")
file.close()

file saved


In [15]:
df=pd.read_csv("/kaggle/input/cleaned-weld-right-data/cleaned_data.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
df.Production.unique(),df["Order Operation No"].unique()
le = preprocessing.LabelEncoder()
col=["Production","Order Operation No"]

df[col[0]]=df[col[0]].values.astype(np.str_)
df[col[1]]=df[col[1]].values.astype(np.str_)
df[col[0]] = le.fit_transform(df[col[0]].values)
df[col[1]] = le.fit_transform(df[col[1]].values)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827534 entries, 0 to 827533
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Employee Code       827534 non-null  int64  
 1   Machine             827534 non-null  object 
 2   Production          827534 non-null  int64  
 3   Order Operation No  827534 non-null  int64  
 4   Date                827534 non-null  object 
 5   Time                827534 non-null  object 
 6   Current             827531 non-null  float64
 7   Humidity            827534 non-null  int64  
 8   Temperature         827534 non-null  int64  
 9   Flow                827534 non-null  float64
 10  Job Temp            827534 non-null  float64
 11  Voltage             827534 non-null  float64
 12  Defect              827534 non-null  object 
 13  label               827534 non-null  int64  
dtypes: float64(4), int64(6), object(4)
memory usage: 88.4+ MB


In [17]:
dftt=transform2(df)
file=open("101.bin","wb+")
p.dump(dftt,file)
print("saved file 2")
file.close()

100%|██████████| 827534/827534 [00:06<00:00, 122961.91it/s]


   Employee Code Machine  Production  Order Operation No        Date  \
0         382617  TWLD23           5                   4  10-09-2022   
1         382617  TWLD23           5                   4  10-09-2022   
2         382617  TWLD23           5                   4  10-09-2022   
3         382617  TWLD23           5                   4  10-09-2022   
4         382617  TWLD23           5                   4  10-09-2022   

       Time  Current  Humidity  Temperature  Flow  ...         Heat  \
0         0     1.13        74           23  0.01  ...      0.00000   
1  27149040     1.82        74           23  0.00  ...  49411.25280   
2  27149677     2.91        74           23  0.00  ...      1.85367   
3  27150166     0.04        74           23  0.00  ...      0.01956   
4  27150696     2.51        74           23  0.00  ...      1.33030   

   lag_Power lag_Delta_Time  lag_Delta_Temperature  lag_Alpha     lag_Heat  \
0      0.000              0                    0.0   0.000000 